<a href="https://colab.research.google.com/github/AfonsoPaula/Youtube_API/blob/main/youtube_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries

import requests
import pandas as pd
import time

import warnings
warnings.filterwarnings("ignore")

In [6]:
# Keys - How to get API key for youtube?
#        -> https://www.slickremix.com/docs/get-api-key-for-youtube/

API_KEY = "**********************************"
CHANNEL_ID = "**********************************"

In [3]:
def get_video_details(video_id):

    # Collecting view, like, dislike and comment count
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_url_video_stats = requests.get(url_video_stats).json()

    view_count = response_url_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_url_video_stats['items'][0]['statistics']['likeCount']
    favorite_count = response_url_video_stats['items'][0]['statistics']['favoriteCount']
    comment_count = response_url_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count, favorite_count, comment_count

In [4]:
def get_videos(df):

    # Make API call
    pageToken = ""
    url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000"+pageToken

    response = requests.get(url).json()
    time.sleep(1)

    for video in response['items']:

        if video['id']['kind'] == "youtube#video":

            video_id = video['id']['videoId']
            video_title = video['snippet']['title']
            video_title = str(video_title).replace("&amp:", " ")
            upload_date = video['snippet']['publishedAt']
            upload_date = str(upload_date).split("T")[0]

            view_count, like_count, favorite_count, comment_count = get_video_details(video_id)

            # Save data in pandas df
            df = df.append({
                'video_id': video_id,
                'video_title': video_title,
                'upload_date': upload_date,
                'view_count': view_count,
                'like_count': like_count,
                'comment_count': comment_count}, ignore_index=True)

    return df

In [5]:
# Main

# Build our DataFrame
df = pd.DataFrame(columns=["video_id","video_title","upload_date", "view_count",
                           "like_count","favorite_count","comment_count"])

df = get_videos(df)

df.head(5)

,video_id,video_title,upload_date,view_count,like_count,favorite_count,comment_count
0,WS0fM1agxTk,Master SQL Subqueries with Multiple Tables,2023-09-20,859,42,NaN,4
1,mqHQwF6_r1E,6 Stages of Data Science Project,2023-09-13,1070,77,NaN,2
2,oJglmpO_esU,3 Data Science Projects To Land A Job,2023-09-06,1837,97,NaN,9
3,4uVxxj07cRE,Is SQL Really the ONLY Language You Need for D...,2023-08-29,1314,62,NaN,7
4,3FRoEyC0hdI,How To Be a Lazy Data Scientist With AI Chrome...,2023-08-16,872,66,NaN,1
